#Парсинг сообщений с сайта woman.ru

In [ ]:
!pip install fake_useragent

In [ ]:
import requests
from pprint import pprint
from fake_useragent import UserAgent
import random
from bs4 import BeautifulSoup
import re
from tqdm.auto import tqdm
import pandas as pd
import time
from datetime import datetime
import json

In [ ]:
session = requests.session()
ua = UserAgent()
headers = {'User-Agent': ua.random}

In [ ]:
import json

def parse_each_topic(topic):
  # выбрала топовые треды в теме
  url_one = f'https://www.woman.ru/{topic}/forum/?sort=all'
  req = session.get(url_one, headers={'User-Agent': ua.random})
  page = req.text
  soup = BeautifulSoup(page, 'html.parser')
  news = soup.find_all('a', {'class': 'list-item__link'})
  threads = []
  # взяла десять тредов с 1 по 20 с шагом 2, чтобы были разные даты создания
  for i in tqdm(news[0:20:2]):
    thread = {}
    link = i.attrs['href']
    try:
      year = int(i.find('time', {'class': 'list-item__date'}).text.split(', ')[0].split()[-1])
      year_for_div = 2024 - year
    except Exception as e:
      print(e)
    name = i.find('h3', {'class': 'list-item__title list-item__title_visited text_weight_medium'}).text
    thread['name'] = name
    url_for_first = f'https://www.woman.ru' + link
    req = session.get(url_for_first, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    num_of = soup.find_all('a', {'class': 'pager__link'})
    num_of_last_page = int(num_of[-2].text)
    # шаг – количество страниц, которое пропускаю, чтобы перейти на следующее
    # нужно, чтобы равномерно выбрать сообщения по дате их отправки
    # беру номер последней страницы и делю на 6, либо на количество лет с дня открытия треда
    # конечно, это не очень хорошо, так как я рассчитываю при такой формуле на то, что данные распределены по страница
    # равномерно в зависимости от года, но это естественно не так
    shag = int(num_of_last_page / max(6, year_for_div))
    comments = []
    for n in range(1, num_of_last_page, shag):
      urlik = ' https://www.woman.ru' + link + f'{n}/'
      time.sleep(random.uniform(1.1, 3.3))
      req = session.get(urlik, headers={'User-Agent': ua.random})
      page = req.text
      soup = BeautifulSoup(page, 'html.parser')
      full_l = soup.find_all('div', {'class': 'card__text'})
      for element in full_l[1:]:
        one_comment = {}
        try:
          list_of_years = element.find('time').text.split(', ')[0].split()
          if len(list_of_years) == 3:
            one_comment['year'] = int(list_of_years[-1])
          else:
            one_comment['year'] = 2023
          one_comment['text'] = element.find('p', {'class': 'card__comment'}).text
          comments.append(one_comment)
        except Exception:
          pass
    thread['comments'] = comments
    threads.append(thread)
  # создаю json-файл для каждого топика
  with open(f"{topic}.json", "w", encoding='utf-8') as final:
    json.dump(threads, final, ensure_ascii=False)
  return final

In [ ]:
topics = ['relations', 'psycho', 'health', 'kids', 'home', 'beauty', 'fashion', 'rest', 'stars']
for top in tqdm(topics):
  parse_each_topic(top)


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

# делаю из каждого файлы один датасет
topics = ['relations', 'psycho', 'health', 'kids', 'home', 'beauty', 'fashion', 'rest', 'stars']
ls = []
df_list = []
for name in topics:
  with open(f'{name}.json') as f:
    data = json.load(f)
  df = pd.json_normalize(data,'comments', ['name'])
  df.insert(0, "topic", [name] * len(df), allow_duplicates=True)
  ls.append(len(df))
  df_list.append(df)
df_final = pd.concat(df_list, axis=0)
df_final.to_csv('woman.ru – 9 topic.csv', index=False)